<a href="https://colab.research.google.com/github/mrwan137/Health-Tweets-clustring/blob/main/tweets_clustring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import math
import string
import re

In [ ]:

def default_number_of_clusters (number_of_clusters):
    number_of_clusters=3
    return number_of_clusters
def chose_number_of_clusters(number_of_clusters):
       print("Enter number of clusters you want")
       choice2=int(input(":"))
       number_of_clusters=choice2
       return number_of_clusters
def number_of_clusters():
    number_of_clusters=0
    flag = False          
    while (flag == False):
           print("Do you want to use default number of clusters?")
           print("1.yes")
           print("2.No")
           choice=input(":")
           choice=int(choice)
           if choice==1:
               number_of_clusters=default_number_of_clusters(number_of_clusters)
               flag=True
           elif choice==2: 
               number_of_clusters=chose_number_of_clusters(number_of_clusters)
               flag = True
           else:
              print("invalid choise")
    return number_of_clusters


def default_number_of_experment (number_of_experment):
    number_of_experment=5
    return number_of_experment
def chose_number_of_experment(number_of_experment):
       print("Enter number of experment you want")
       choice2=int(input(":"))
       number_of_experment=choice2
       return number_of_experment
def number_of_experment():
    number_of_experment=0
    flag = False          
    while (flag == False):
           print("Do you want to use default number of experment?")
           print("1.yes")
           print("2.No")
           choice=input(":")
           choice=int(choice)
           if choice==1:
               number_of_experment=default_number_of_experment(number_of_experment)
               flag=True
           elif choice==2: 
               number_of_experment=chose_number_of_experment(number_of_experment)
               flag = True
           else:
              print("invalid choise")
    return number_of_experment



         
         
files=["data/cbchealth.txt","data/cnnhealth.txt","data/everydayhealth.txt","data/foxnewshealth.txt"]
def default (file_location):
    #print("data/bbchealth.txt")
    file_location="data/bbchealth.txt"
    return file_location
def chose_file(file_location):
       for i in files:
          index=files.index(i,0,15)+1
          print(index," ",i)
       print("Enter your file number")
       choice2=int(input(":"))-1
       #print(files[choice2])
       file_location=files[choice2]
       return file_location
def menu():
    file_location=""
    flag = False          
    while (flag == False):
           print("Do you want to use default file?")
           print("1.yes")
           print("2.No")
           choice=input(":")
           choice=int(choice)
           if choice==1:
               file_location=default(file_location)
               flag=True
           elif choice==2: 
               file_location=chose_file(file_location)
               flag = True
           else:
              print("invalid choise")
    return file_location
         

In [ ]:

def pre_process_tweets(location):
  f = open(location, "r", encoding="utf8")
  tweets = list(f)

  list_of_tweets = []
  
  for i in range(len(tweets)):
    tweets[i] = tweets[i].strip('\n')
    
    # Remove  tweet id and timestamp
    tweets[i] = tweets[i][50:] #[50:]
    
    # Remove words with symbol @
    tweets[i] = " ".join(filter(lambda x: x[0] != '@', tweets[i].split()))
    
    # Remove any hash-tags symbols
    tweets[i] = tweets[i].replace('#', '')
    
    # Remove any URL
    tweets[i] = re.sub(r"http\S+", "", tweets[i])
    tweets[i] = re.sub(r"www\S+", "", tweets[i])
    tweets[i] = tweets[i].strip()
    """
    tweet_len = len(tweets[i])
          if tweet_len > 0:
              if tweets[i][len(tweets[i]) - 1] == ':':
                  tweets[i] = tweets[i][:len(tweets[i]) - 1]
    """
    
    # Convert every word to lowercase
    tweets[i] = tweets[i].lower()
    
    # convert each tweet from string type to as list<string> using " " as a delimiter
    list_of_tweets.append(tweets[i])

    f.close()
  return list_of_tweets


In [ ]:
 
def jaccard(list1, list2):
    list1=re.split('\s+', list1)
    list2=re.split('\s+',list2)
    
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float (1.0-(float(intersection))/union)
      



def k_means(tweets,k=3):
    
    centroids = []
    means = []
    clusters = []
    for i in range(k):
        means.append(0.0)
        clusters.append({})
        centroids.append(tweets[i])


    for gg in range(50):
        means = []
        clusters = []
        for i in range(k):
            means.append(0.0)
            clusters.append({})

        for i in range(len(tweets)):
            old_jaccard = 2
            best_centroid_index = 0
            for j in range(len(centroids)):
               new_jacard = jaccard(tweets[i], centroids[j])
               if new_jacard <= old_jaccard:
                   old_jaccard = new_jacard
                   best_centroid_index=j
                   
            clusters[best_centroid_index][str(old_jaccard)] = tweets[i]
            means[best_centroid_index] =  means[best_centroid_index] + old_jaccard
         
           
        for i in range(k):
            means[i] = means[best_centroid_index] / len(clusters[i])
            
        new_centroid = []
        for i in range(k):
            new_centroid.append(update_centroid(clusters[i], means[i]))
         
        
        if new_centroid == centroids:
            break
        else:
            centroids = new_centroid
    
    return clusters, sse(clusters, centroids, means)
    
def update_centroid(cluster, mean): 
    cluster = dict(cluster)
    mean = float(mean)
    
    f = lambda a, l:min(l, key=lambda x: abs(x-a))
    numbers = list(np.float_(list(cluster.keys())))
    closest = f(mean, numbers)
    
    new_centroid = cluster[str(closest)]
    

    return new_centroid






def sse(clusters, centroids, means):
   clusters = list(clusters)
   sums = 0.0
   for i in range(len(centroids)):	
      dists = list(np.float_(list(clusters[i].keys())))
      for j in range(len(dists)):
          sums =sums + math.pow(means[i] - dists[j],2)

   return sums



In [ ]:

### main run
url = menu ()
k = number_of_clusters()
e = number_of_experment()
tweets = pre_process_tweets(url)
exp = []
errors = []
for i in range(e):
  clusters, error = k_means(tweets, i+1)
  print(error)
  exp.append(i+1)
  errors.append(error)
  

plt.plot(exp, errors)
plt.show()
